# HMM for decryption


In [9]:
from src.CipherUtils import CipherGenerator
from src.CipherUtils import TextEncoder
from src.ProbabilityMatrix import ProbabilityMatrix
from src.CipherUtils import TextPreProcessor

from src.HMM_utils import map_alphabet_to_numbers, string_to_numbers
from src.HMM_utils import find_mapping, numbers_to_string, invert_mapping

from src.HMM_functions import Baum_Welch

import numpy as np

In [10]:
# RICORDATI DI LEVARE LO SPAZIO
hidden_sequence = "people of western europe a landing was made this morning on the coast of france by troops kangaroo jokes quasi vile xilophone zenit "
hidden_sequence = "in germany it seems to be pretty much automatic pretty much all the time in france and spain it all depends presumably on social subtleties that you have to be french or spanish to understand in italy why would you even bother when and how much to tip is a question that has been vexing visitors to europe for as long as people have been travelling around the continent outside their own country it seems even europeans don t know the answer according to new polling by yougov in six eu countries britain and the us where as most visitors know but may be reluctant to acknowledge gratuities may make up more than half your waitperson s income europeans are deeply divided on tipping in restaurants for example of respondents in germany told the pollster they typically tipped almost the same as the us in the uk where an optional service charge of about is usually included said they left a gratuity the figure in spain where service is often included in restaurant bills but diners can leave optional tips was while in france where every price on a restaurant menu already includes for service of people said they generally tipped on top even in sweden where tips are generally not expected the figure was but only of italians said they would typically leave a gratuity after a meal out with a rather greater proportion admitting they never left a cent a startling of respondents in the us however and of germans by far the most in europe confessed they would tip sometimes or often even if the service was terrible indicating that for some tipping is not about quality of service at all the findings of the survey will come as a surprise in germany a country that does not generally think of itself as a nation of happy distributors"
# hidden_sequence = " hello banana xilophone key queue zebra cock pussy tits dandy fart though jolly world mum "

cipher_generator = CipherGenerator()
cipher = cipher_generator.generate_cipher()
encoder = TextEncoder()
observed_sequence = encoder.encode_text(hidden_sequence, cipher=cipher)

print(hidden_sequence)
print(observed_sequence)

ri ecnyhiu rw vccyv wo mc kncwwu yqzg hqwoyhwrz kncwwu yqzg hss wgc wryc ri fnhizc hib vkhri rw hss bckcibv kncvqyhmsu oi vozrhs vqmwscwrcv wghw uoq ghxc wo mc fncizg on vkhirvg wo qibcnvwhib ri rwhsu tgu toqsb uoq cxci mowgcn tgci hib got yqzg wo wrk rv h lqcvwroi wghw ghv mcci xcarie xrvrwonv wo cqnokc fon hv soie hv kcoksc ghxc mcci wnhxcssrie hnoqib wgc zoiwriciw oqwvrbc wgcrn oti zoqiwnu rw vccyv cxci cqnokchiv boi w jiot wgc hivtcn hzzonbrie wo ict kossrie mu uoqeox ri vra cq zoqiwnrcv mnrwhri hib wgc qv tgcnc hv yovw xrvrwonv jiot mqw yhu mc ncsqzwhiw wo hzjiotscbec enhwqrwrcv yhu yhjc qk yonc wghi ghsf uoqn thrwkcnvoi v rizoyc cqnokchiv hnc bccksu brxrbcb oi wrkkrie ri ncvwhqnhiwv fon cahyksc of ncvkoibciwv ri ecnyhiu wosb wgc kossvwcn wgcu wukrzhssu wrkkcb hsyovw wgc vhyc hv wgc qv ri wgc qj tgcnc hi okwroihs vcnxrzc zghnec of hmoqw rv qvqhssu rizsqbcb vhrb wgcu scfw h enhwqrwu wgc freqnc ri vkhri tgcnc vcnxrzc rv ofwci rizsqbcb ri ncvwhqnhiw mrssv mqw bricnv zhi schxc okwroih

In [4]:
# List of text file paths to build our corpus (where we learn the transitions probs)
file_paths = [
    "texts/moby_dick.txt",
    "texts/shakespeare.txt",
    "texts/james-joyce-a-portrait-of-the-artist-as-a-young-man.txt",
    "texts/james-joyce-dubliners.txt",
    "texts/james-joyce-ulysses.txt",
]

texts = []
for file_path in file_paths:
    with open(file_path, "r") as file:
        texts.append(file.read())

corpus = "".join(texts)
alphabet = list("abcdefghijklmnopqrstuvwxyz ")

preprocessor = TextPreProcessor()
corpus = preprocessor.lower(text=corpus)
corpus = preprocessor.remove_additional_spaces(text=corpus)
corpus = preprocessor.remove_unknown_chars(
    text=corpus, unknown_chars=preprocessor.unknown_chars(corpus)
)

# compute probabilities
p = ProbabilityMatrix(corpus)
p.compute_probability_matrix()
p.compute_normalized_matrix()
# p.compute_probability_table()

#### Following line to check consistency of probability_table (the dictionary) and probability_matrix (the matrix)


In [15]:
pm.matrix[-1,] # last entry should be 0

array([1.80999782e-02, 9.22150531e-03, 7.18584308e-03, 6.82098237e-03,
       3.75680960e-03, 7.32056810e-03, 4.03529608e-03, 1.29248004e-02,
       1.19011015e-02, 8.04594734e-04, 1.60062246e-03, 6.07119302e-03,
       9.71968276e-03, 4.78543749e-03, 1.00901766e-02, 5.64976658e-03,
       5.00407226e-04, 3.27975627e-03, 1.55191960e-02, 2.64728801e-02,
       1.90821330e-03, 1.18694153e-03, 1.20175189e-02, 6.10252715e-06,
       4.03494401e-03, 3.42680371e-05, 4.02150671e-02])

In [16]:
p1 = ProbabilityMatrix(corpus)
p1.compute_probability_table()

In [5]:
dict(sorted(p1.probability_table.items(), key=lambda x: x[0]))

{'  ': 0,
 ' a': 0.01810018209207315,
 ' b': 0.00922160919820809,
 ' c': 0.007185924033947333,
 ' d': 0.006821059213488051,
 ' e': 0.003756851924265846,
 ' f': 0.007320650574277746,
 ' g': 0.004035341541168347,
 ' h': 0.012924946050705061,
 ' i': 0.011901235587898271,
 ' j': 0.0008046037983495764,
 ' k': 0.0016006404909115917,
 ' l': 0.006071261420656161,
 ' m': 0.009719792267635446,
 ' n': 0.0047854914068408225,
 ' o': 0.010090290253544083,
 ' p': 0.00564983023047627,
 ' q': 0.0005004128640843924,
 ' r': 0.0032797932252735724,
 ' s': 0.015519370812973743,
 ' t': 0.026473178386858576,
 ' u': 0.0019082347959690946,
 ' v': 0.0011869549032245646,
 ' w': 0.01201765434051828,
 ' x': 6.102595903468199e-06,
 ' y': 0.004034989468327762,
 ' z': 3.4268423150244504e-05,
 'a ': 0.004482239333483864,
 'aa': 2.370623793270339e-05,
 'ab': 0.0009797013577337024,
 'ac': 0.001917388689824297,
 'ad': 0.002408647660053487,
 'ae': 0.00014388043418561561,
 'af': 0.0004848043014851371,
 'ag': 0.0011447061623

In [6]:
def compare(prob1, prob2, acc=10):
    # prob1 deve essere matrix
    # prob2 deve essere dict
    i = 0  # number of errors (should be 0 :))
    errors = []
    error_letter = []
    for x in "abcdefghijklmnopqrstuvwxyz ":
        for y in "abcdefghijklmnopqrstuvwxyz ":
            if round(prob1.get_probability_mat(x, y), acc) != round(
                prob2.get_probability(x + y), acc
            ):
                i += 1
                errors.append(
                    prob1.get_probability_mat(x, y) - prob2.get_probability(x + y)
                )
                error_letter.append([x, y])

            elif round(prob1.get_probability_mat(y, x), acc) != round(
                prob2.get_probability(y + x), acc
            ):
                i += 1
                errors.append(
                    prob1.get_probability_mat(x, y) - prob2.get_probability(x + y)
                )
                error_letter.append([y, x])

    return i, errors, error_letter


i, errors, error_letter = compare(p, p1)
error_letter

[]

#### Go on


In [8]:
# translate the hidden and observed sequences into numeric list
hidden_ = string_to_numbers(hidden_sequence, mapping=map_alphabet_to_numbers())
observed_ = string_to_numbers(observed_sequence, mapping=map_alphabet_to_numbers())

In [10]:
B_start = np.zeros((27, 27)) + 1 / 26
B_start[:, -1] = np.zeros(27)
B_start[-1, :] = np.zeros(27)
B_start[-1, -1] = 1

emission = Baum_Welch(
    A=p.normalized_matrix,
    B_start=B_start,
    pi=p.probability_matrix[-1, :],
    observed=observed_,
    maxIter=100,
)

In [11]:
emission.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [12]:
# go back to letters to see the results
mapping = find_mapping(emission.argmax(axis=1))
numbers_to_string(observed_sequence, invert_mapping(mapping))

'in eqrmany it sqqms to bq prqtty muzh automatiz prqtty muzh all thq timq in franzq and spain it all dqpqnds prqsumably on sozial subtlqtiqs that you havq to bq frqnzh or spanish to undqrstand in italy why would you qvqn bothqr whqn and how muzh to tip is a luqstion that has bqqn vqxine visitors to quropq for as lone as pqoplq havq bqqn travqlline around thq zontinqnt outsidq thqir own zountry it sqqms qvqn quropqans don t know thq answqr azzordine to nqw polline by youeov in six qu zountriqs britain and thq us whqrq as most visitors know but may bq rqluztant to azknowlqdeq eratuitiqs may makq up morq than half your waitpqrson s inzomq quropqans arq dqqply dividqd on tippine in rqstaurants for qxamplq of rqspondqnts in eqrmany told thq pollstqr thqy typizally tippqd almost thq samq as thq us in thq uk whqrq an optional sqrvizq zhareq of about is usually inzludqd said thqy lqft a eratuity thq fieurq in spain whqrq sqrvizq is oftqn inzludqd in rqstaurant bills but dinqrs zan lqavq option

In [ ]:
from src.HMM_functions import Viterbi

pmax, phi = Viterbi(A=pm.matrix, B=emission, observed=observed_) # Questo va aggiustato

**Encrypted text:** \
it eavfqto in baafb nl sa zvanno fdrk qdnlfqnir zvanno fdrk qxx nka nifa it gvqtra qth bzqit in qxx hazathb zvabdfqsxo lt blriqx bdsnxaniab nkqn old kqca nl sa gvatrk lv bzqtibk nl dthavbnqth it inqxo uko uldxh old acat slnkav ukat qth klu fdrk nl niz ib q pdabnilt nkqn kqb saat cajite cibinlvb nl advlza glv qb xlte qb zalzxa kqca saat nvqcaxxite qvldth nka rltnitatn ldnbiha nkaiv lut rldtnvo in baafb acat advlzaqtb hlt n ytlu nka qtbuav qrrlvhite nl tau zlxxite so oldelc it bij ad rldtnviab svinqit qth nka db ukava qb flbn cibinlvb ytlu sdn fqo sa vaxdrnqtn nl qrytluxahea evqndiniab fqo fqya dz flva nkqt kqxg oldv uqinzavblt b itrlfa advlzaqtb qva haazxo hicihah lt nizzite it vabnqdvqtnb glv ajqfzxa lg vabzlthatnb it eavfqto nlxh nka zlxxbnav nkao nozirqxxo nizzah qxflbn nka bqfa qb nka db it nka dy ukava qt lzniltqx bavcira rkqvea lg qsldn ib dbdqxxo itrxdhah bqih nkao xagn q evqndino nka giedva it bzqit ukava bavcira ib lgnat itrxdhah it vabnqdvqtn sixxb sdn hitavb rqt xaqca lzniltqx nizb uqb ukixa it gvqtra ukava acavo zvira lt q vabnqdvqtn fatd qxvaqho itrxdhab glv bavcira lg zalzxa bqih nkao eatavqxxo nizzah lt nlz acat it buahat ukava nizb qva eatavqxxo tln ajzarnah nka giedva uqb sdn ltxo lg inqxiqtb bqih nkao uldxh nozirqxxo xaqca q evqndino qgnav q faqx ldn uink q vqnkav evaqnav zvlzlvnilt qhfinnite nkao tacav xagn q ratn q bnqvnxite lg vabzlthatnb it nka db kluacav qth lg eavfqtb so gqv nka flbn it advlza rltgabbah nkao uldxh niz blfanifab lv lgnat acat ig nka bavcira uqb navvisxa ithirqnite nkqn glv blfa nizzite ib tln qsldn pdqxino lg bavcira qn qxx nka githiteb lg nka bdvcao uixx rlfa qb q bdvzviba it eavfqto q rldtnvo nkqn hlab tln eatavqxxo nkity lg inbaxg qb q tqnilt lg kqzzo hibnvisdnlvb


**runnato con 200 iterazione di BW:** \
 in germany it seems to je pretty muzh automatiz pretty muzh all the time in franze and spain it all depends presumajly on sozial sujtleties that you have to je frenzh or spanish to understand in italy why would you even jother when and how muzh to tip is a question that has jeen vexing visitors to europe for as long as people have jeen travelling around the zontinent outside their own zountry it seems even europeans don t onow the answer azzording to new polling jy yougov in six eu zountries jritain and the us where as most visitors onow jut may je reluztant to azonowledge gratuities may maoe up more than half your waitperson s inzome europeans are deeply divided on tipping in restaurants for example of respondents in germany told the pollster they typizally tipped almost the same as the us in the uo where an optional servize zharge of ajout is usually inzluded said they left a gratuity the figure in spain where servize is often inzluded in restaurant jills jut diners zan leave optional tips was while in franze where every prize on a restaurant menu already inzludes for servize of people said they generally tipped on top even in sweden where tips are generally not expezted the figure was jut only of italians said they would typizally leave a gratuity after a meal out with a rather greater proportion admitting they never left a zent a startling of respondents in the us however and of germans jy far the most in europe zonfessed they would tip sometimes or often even if the servize was terrijle indizating that for some tipping is not ajout quality of servize at all the findings of the survey will zome as a surprise in germany a zountry that does not generally thino of itself as a nation of happy distrijutors'
